### イメージ(社内の閉鎖空間)であるデータベースに対してのチャットボットを作成していきます。

In [4]:
# # イメージ:以下のようなquestionからdocuments内にある適切な回答を選択
# question = "2023年の第1事業部の売り上げはどのぐらい？"

# documents = [
#     "2023年上期売上200億円, 下期売上300億円",
#     "2023年第1事業部売上300億円, 第2事業部売上150億円, 第3事業部売上50億円",
#     "2024年は全社で1000億円の売上を目指す"
# ]

In [5]:
!pip install openai==1.25.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [6]:
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [7]:
# OpenAIをimportする
import os
from openai import OpenAI
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity #ベクトル間の類似度を計算
from google.colab import userdata

In [8]:

# 環境変数からAPIキーを設定
os.environ["OPEN_API_KEY"] = userdata.get('OPEN_API_KEY')


# OpenAIクライアントを初期化
client = OpenAI(api_key=os.environ["OPEN_API_KEY"])

text = "2023年上期売上200億円, 下期売上300億円"

# テキストをベクトル化
  #client.embeddingsでembeddingsAPIをcallする
def vectorize_text(text):
  response = client.embeddings.create(
      input = text,
      model = "text-embedding-3-small"
  )
  return response.data[0].embedding

#:以下のようにquestionからdocuments内にある適切な回答を選択
question = "2023年の第1事業部の売り上げはどのぐらい？"

documents = [
    "2023年上期売上200億円, 下期売上300億円",
    "2023年第1事業部売上300億円, 第2事業部売上150億円, 第3事業部売上50億円",
    "2024年は全社で1000億円の売上を目指す"
]

# テキストを一つ一つ取り出し、ベクトル化処理に入れ込む
vectors = [vectorize_text(doc) for doc in documents]
question_vector = vectorize_text(question)

print(vectors)
print(question_vector)




[[0.04189140722155571, -0.02258175052702427, 0.03816724941134453, -0.006398879922926426, 0.027339201420545578, 0.022667856886982918, 0.025918424129486084, 0.08068293333053589, -0.008508519269526005, -0.023701149970293045, 0.00580688938498497, -0.015219539403915405, -0.08877705782651901, -0.032419558614492416, 0.026801029220223427, -0.015144195407629013, -0.027640579268336296, -0.02238800749182701, -0.05575475096702576, -0.019804775714874268, 0.004964648280292749, 0.0008079325780272484, 0.019331183284521103, 0.00036293058656156063, -0.028587764129042625, 0.024992765858769417, -0.017684374004602432, -0.034292399883270264, 0.026564233005046844, 0.0024096707347780466, 0.04623984545469284, -0.03810266777873039, 0.000636726210359484, 0.0032909295987337828, -0.031343210488557816, -0.002661266829818487, 0.009175853803753853, 0.04094422236084938, -0.02652117796242237, 0.018620794638991356, 0.0014719038736075163, 0.01642504706978798, 0.01934194751083851, -0.006226664409041405, 0.0273392014205455

In [9]:
# ベクトル同士の類似度を確認(2次元)
 #ここではquestionとdocumentsの2番目の要素を比べてる
cosine_similarity([question_vector], [vectors[2]])[0][0]
# 全類似度を比べ最も値が高いものを取得したい

0.6012472253616964

In [10]:
# documentsの中でqusetionと最も高い類似度を格納
max_similarity = 0
most_similar_index = 0

for index, vector in enumerate(vectors):
  similarity = cosine_similarity([question_vector], [vector])[0][0]
  print(documents[index], ":", similarity) #処理の流れ確認
  if similarity > max_similarity:
    max_similarity = similarity
    most_similar_index = index

# print(documents[most_similar_index])

2023年上期売上200億円, 下期売上300億円 : 0.5859857108413727
2023年第1事業部売上300億円, 第2事業部売上150億円, 第3事業部売上50億円 : 0.8410304604487016
2024年は全社で1000億円の売上を目指す : 0.6012472253616964


In [11]:
print(documents[most_similar_index])

2023年第1事業部売上300億円, 第2事業部売上150億円, 第3事業部売上50億円


###:以下のようにquestionからdocuments内にある適切な回答を選択
question = "2023年の第1事業部の売り上げはどのぐらい？"

documents = [
    "2023年上期売上200億円, 下期売上300億円",
    "2023年第1事業部売上300億円, 第2事業部売上150億円, 第3事業部売上50億円",
    "2024年は全社で1000億円の売上を目指す"
]

In [12]:
# 実際にOpenAIに上記の情報をインプットさせ出力させる
prompt = f''' 以下の質問に以下情報から答えてください。
[ユーザへの質問]
{question}

[情報]
{documents[most_similar_index]}
'''

response = client.completions.create(
    model = "gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens = 200
)

print(response.choices[0].text)


[回答]
2023年の第1事業部の売り上げは300億円です。


### 本題:特定のURLの情報(Webページ)にRAGを使ってアクセスし、チャットボットから回答を得る

In [13]:
import requests
from bs4 import BeautifulSoup #ウェブスクレイピングでHTMLデータを解析し、データ抽出
from openai import OpenAI
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 対象のURLを取得
url = "https://toukei-lab.com/achademy/?page_id=1619"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser") #対象urlのhtmlを取得
print(soup)

# テキストの情報だけを取得してくる
text_nodes = soup.find_all("div")
len(text_nodes) #divで囲まれてる箇所何か所あるか確認
for t in text_nodes:
  print(t.text) #t.textでtext属性のものだけを取得できる

# 上記テキスト情報をリストへ格納
t_all = []
for t in text_nodes:
  t_all.append(t.text.replace("\n","").replace("\t","")) #\n,\tを""に変換"
  print(t_all)

#文字列をまずはすべて結合させる
joined_text =  "".join(t_all)
joined_text

<!DOCTYPE html>

<html data-loaded="false" data-scrolled="false" data-spmenu="closed" lang="ja">
<head>
<meta charset="utf-8"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, viewport-fit=cover" name="viewport"/>
<meta content="max-image-preview:large" name="robots">
<script>
MathJax = {
  tex: {
    inlineMath: [['$','$'],['\\(','\\)']], 
    processEscapes: true
  },
  options: {
    ignoreHtmlClass: 'tex2jax_ignore|editor-rich-text'
  }
};

</script>
<link href="//cdn.jsdelivr.net" rel="dns-prefetch"/>
<link href="https://toukei-lab.com/achademy/?feed=rss2" rel="alternate" title="スタアカ » フィード" type="application/rss+xml"/>
<link href="https://toukei-lab.com/achademy/?feed=comments-rss2" rel="alternate" title="スタアカ » コメントフィード" type="application/rss+xml"/>
<!-- SEO SIMPLE PACK 3.2.1 -->
<title>プラン一覧 | スタアカ</title>
<meta content="noindex" name="robots"/>
<meta content="プラン一覧 ライトプラン ライトプラン128

'MENUTOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 ウォルマートの需要予測コンペに挑戦してAmazonギフト券をゲットしよう！【12月31日まで】AIデータサイエンス特化スクール TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるライトプランでは当サービス「スタアカ」の全コースが全て受講し放題です。※1コースごとの価格ではなくライトプランに登録いただくと全コースが受講可能になりますデジタルトランスフォーメーション(DX)概要コースデータサイエンティスト概要コースPython基礎コースデータ加工集計可視化コース統計学概要コース機械学習概要コース機械学習実装コースディープラーニングコースSQL基礎コースなどなど(※1ヶ月ごとに新しいカリキュラム追加)※決済いただいた日付から1ヶ月間のご受講が始まります。次回の請求はご受講開始から1ヶ月後です。プレミアムプランプレミアムプラン\\ただいまセール中/149,800 円129,800 円/ 買い切り全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるプレミアムプランは確実に実力を付けるためのプランです。・スタアカ専用QAフォームでの24時間以内の返信対応（購入後ずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）含む現役データサイエンティストとのマンツーマンメンタリング（

In [14]:
# 改行・タブを""に変換したうえですべての文字列を結合し結合
joined_text

'MENUTOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 ウォルマートの需要予測コンペに挑戦してAmazonギフト券をゲットしよう！【12月31日まで】AIデータサイエンス特化スクール TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるライトプランでは当サービス「スタアカ」の全コースが全て受講し放題です。※1コースごとの価格ではなくライトプランに登録いただくと全コースが受講可能になりますデジタルトランスフォーメーション(DX)概要コースデータサイエンティスト概要コースPython基礎コースデータ加工集計可視化コース統計学概要コース機械学習概要コース機械学習実装コースディープラーニングコースSQL基礎コースなどなど(※1ヶ月ごとに新しいカリキュラム追加)※決済いただいた日付から1ヶ月間のご受講が始まります。次回の請求はご受講開始から1ヶ月後です。プレミアムプランプレミアムプラン\\ただいまセール中/149,800 円129,800 円/ 買い切り全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるプレミアムプランは確実に実力を付けるためのプランです。・スタアカ専用QAフォームでの24時間以内の返信対応（購入後ずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）含む現役データサイエンティストとのマンツーマンメンタリング（

上記まででWebスクレイピング作業が完了
-------------------------------------------------

In [15]:
# 上記全文字列を分割。
len(joined_text)
joined_text

'MENUTOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 ウォルマートの需要予測コンペに挑戦してAmazonギフト券をゲットしよう！【12月31日まで】AIデータサイエンス特化スクール TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるライトプランでは当サービス「スタアカ」の全コースが全て受講し放題です。※1コースごとの価格ではなくライトプランに登録いただくと全コースが受講可能になりますデジタルトランスフォーメーション(DX)概要コースデータサイエンティスト概要コースPython基礎コースデータ加工集計可視化コース統計学概要コース機械学習概要コース機械学習実装コースディープラーニングコースSQL基礎コースなどなど(※1ヶ月ごとに新しいカリキュラム追加)※決済いただいた日付から1ヶ月間のご受講が始まります。次回の請求はご受講開始から1ヶ月後です。プレミアムプランプレミアムプラン\\ただいまセール中/149,800 円129,800 円/ 買い切り全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるプレミアムプランは確実に実力を付けるためのプランです。・スタアカ専用QAフォームでの24時間以内の返信対応（購入後ずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）含む現役データサイエンティストとのマンツーマンメンタリング（

### 大規模言語モデル(LAG)に上記文字列を読み込ませるイメージ記載
- 「MENUTOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミア, 」のように","区切りで400文字ごとに切り取り
- LAGに読み込ませるうえで前段の文章を「一部重複」させる形で読み込ませる必要がある
(ex)「MENUTOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミア,チュートリアルプラン一覧プレミアプランを詳しく知るオーダーメイド,」

In [16]:
# 400文字で一区切り、50文字戻り350+50で区切っていく
chunk_size = 400
overlap = 50
chunks = []
start = 0

while start + chunk_size <= len(joined_text): #この文字数までは回し続ける
  chunks.append(joined_text[start:start + chunk_size]) #最初は0:400文字まではいってる
  start += (chunk_size - overlap)
#上記処理では最後が例えばstart:1100だと残り分を残したまま処理が終わってしまうため残り分を算出
if start < len(joined_text):
  chunks.append(joined_text[-chunk_size:]) #つまり最後から400文字まで取得する処理を入れる